# Retrain a detection model for Edge TPU with quant-aware training

## Import TensorFlow

In [ ]:
%tensorflow_version 1.x

In [ ]:
import tensorflow as tf
print(tf.__version__)

## Clone the model and training repos

In [ ]:
! git clone https://github.com/tensorflow/models.git

In [ ]:
! cd models && git checkout f788046ca876a8820e05b0b48c1fc2e16b0955bc

## Import dependencies

For details, see https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md

In [ ]:
! apt-get install -y python python-tk
! pip install Cython contextlib2 pillow lxml jupyter matplotlib

In [ ]:
# Get protoc 3.0.0, rather than the old version already in the container
! wget https://www.github.com/google/protobuf/releases/download/v3.0.0/protoc-3.0.0-linux-x86_64.zip
! unzip protoc-3.0.0-linux-x86_64.zip -d proto3
! mkdir -p local/bin && mkdir -p local/include
! mv proto3/bin/* local/bin
! mv proto3/include/* local/include
! rm -rf proto3 protoc-3.0.0-linux-x86_64.zip

In [ ]:
# Install pycocoapi
! git clone --depth 1 https://github.com/cocodataset/cocoapi.git
! (cd cocoapi/PythonAPI && make -j8)
! cp -r cocoapi/PythonAPI/pycocotools/ models/research/
! rm -rf cocoapi

In [ ]:
# Run protoc on the object detection repo (generate .py files from .proto)
% cd models/research/
! ../../local/bin/protoc object_detection/protos/*.proto --python_out=.

In [ ]:
import os
os.environ['PYTHONPATH'] += ":/content/models/research:/content/models/research/slim"

Just to verify everything is correctly set up:

In [ ]:
! python object_detection/builders/model_builder_test.py

## Prepare data

In [ ]:
# Install Kaggle API
!pip install -q kaggle
!pip install -q kaggle-cli

In [ ]:
# only for google colab
import os
os.environ['KAGGLE_USERNAME'] = "<username>" 
os.environ['KAGGLE_KEY'] = "<key>"

In [ ]:
!kaggle datasets download -d tannergi/microcontroller-detection --unzip

In [ ]:
!mv "Microcontroller Detection" microcontroller-detection

In [ ]:
!wget https://raw.githubusercontent.com/TannerGilbert/Tensorflow-Object-Detection-API-Train-Model/master/generate_tfrecord.py

In [ ]:
!wget https://raw.githubusercontent.com/TannerGilbert/Tensorflow-Object-Detection-API-Train-Model/master/training/labelmap.pbtxt

In [ ]:
!python generate_tfrecord.py --csv_input=microcontroller-detection/train_labels.csv --image_dir=microcontroller-detection/train --output_path=train.record
!python generate_tfrecord.py --csv_input=microcontroller-detection/test_labels.csv --image_dir=microcontroller-detection/test --output_path=test.record

In [ ]:
train_record_path = 'train.record'
test_record_path = 'test.record'
labelmap_path = 'labelmap.pbtxt'

## Download model

In [ ]:
!wget http://download.tensorflow.org/models/tflite_11_05_08/mobilenet_v2_1.0_224_quant.tgz
!tar -xf mobilenet_v2_1.0_224_quant.tgz

## Configuring training

In [ ]:
!wget https://raw.githubusercontent.com/google-coral/tutorials/master/docker/object_detection/scripts/configs/pipeline_mobilenet_v2_ssd_retrain_whole_model.config

In [ ]:
batch_size = 16
num_steps = 3000
num_eval_steps = 500

In [ ]:
fine_tune_checkpoint = 'mobilenet_v2_1.0_224_quant.ckpt'
base_config_path = 'pipeline_mobilenet_v2_ssd_retrain_whole_model.config'

In [ ]:
# edit configuration file (from https://colab.research.google.com/drive/1sLqFKVV94wm-lglFq_0kGo2ciM0kecWD)

import re

with open(base_config_path) as f:
    config = f.read()

with open('model_config.config', 'w') as f:
  
  # Set labelmap path
  config = re.sub('label_map_path: ".*?"', 
             'label_map_path: "{}"'.format(labelmap_path), config)
  
  # Set fine_tune_checkpoint path
  config = re.sub('fine_tune_checkpoint: ".*?"',
                  'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), config)
  
  # Set train tf-record file path
  config = re.sub('(input_path: ".*?)(DATASET_DIR_TO_CONFIGURE/pet_faces_train.record-\?\?\?\?\?-of-00010)(.*?")', 
                  'input_path: "{}"'.format(train_record_path), config)
  
  # Set test tf-record file path
  config = re.sub('(input_path: ".*?)(DATASET_DIR_TO_CONFIGURE/pet_faces_val.record-\?\?\?\?\?-of-00010)(.*?")', 
                  'input_path: "{}"'.format(test_record_path), config)
  
  # Set number of classes.
  config = re.sub('num_classes: [0-9]+',
                  'num_classes: {}'.format(4), config)
  
  # Set batch size
  config = re.sub('batch_size: [0-9]+',
                  'batch_size: {}'.format(batch_size), config)
  
  # Set training steps
  config = re.sub('num_steps: [0-9]+',
                  'num_steps: {}'.format(num_steps), config)
  
  # Set fine-tune checkpoint type to detection
  config = re.sub('fine_tune_checkpoint_type: "classification"', 
             'fine_tune_checkpoint_type: "{}"'.format('detection'), config)
  
  f.write(config)

In [ ]:
%cat model_config.config

In [ ]:
model_dir = 'training/'
pipeline_config_path = 'model_config.config'

## Perform transfer-learning

The following script takes several hours to finish in Colab. (You can shorten by reducing the steps, but that reduces the final accuracy.)

If you didn't already select "Run all" then you should run all remaining cells now. That will ensure the rest of the notebook completes while you are away, avoiding the chance that the Colab runtime times-out and you lose the training data before you download the model.

In [ ]:
!python object_detection/model_main.py \
  --pipeline_config_path={pipeline_config_path} \
  --model_dir={model_dir} \
  --alsologtostderr

## Compile for the Edge TPU

In [ ]:
output_directory = 'inference_graph'

!python object_detection/export_tflite_ssd_graph.py \
  --pipeline_config_path {pipeline_config_path} \
  --trained_checkpoint_prefix {model_dir}model.ckpt-1000 \
  --output_directory {output_directory} \
  --add_postprocessing_op true

In [ ]:
!tflite_convert \
  --output_file {output_directory}/output_tflite_graph.tflite \
  --graph_def_file {output_directory}/tflite_graph.pb \
  --inference_type QUANTIZED_UINT8 \
  --input_arrays normalized_input_image_tensor \
  --output_arrays TFLite_Detection_PostProcess,TFLite_Detection_PostProcess:1,TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3 \
  --mean_values 128 \
  --std_dev_values 128 \
  --input_shapes 1,300,300,3 \
  --change_concat_input_ranges false \
  --allow_nudging_weights_to_use_fast_gemm_kernel true \
  --allow_custom_ops

In [ ]:
ls -lah {output_directory}

In [ ]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

! sudo apt-get update

! sudo apt-get install edgetpu-compiler	

In [ ]:
! edgetpu_compiler {output_directory}/output_tflite_graph.tflite -o {output_directory}

In [ ]:
ls -lah {output_directory}

Download the files:

In [ ]:
from google.colab import files

files.download(f'{output_directory}/output_tflite_graph_edgetpu.tflite')

If you get a "Failed to fetch" error here, it's probably because the files weren't done saving. So just wait a moment and try again.

Also look out for a browser popup that might need approval to download the files.

## Run the model on the Edge TPU




You can now run the model on your Coral device with acceleration on the Edge TPU.

To get started, try using [this code for object detection with the TensorFlow Lite API](https://github.com/google-coral/tflite/tree/master/python/examples/detection). Just follow the instructions on that page to set up your device, copy the `output_tflite_graph_edgetpu.tflite` and `labels.txt` files to your Coral Dev Board or device with a Coral Accelerator, and pass it a photo to see the detected objects.

Check out more examples for running inference at [coral.ai/examples](https://coral.ai/examples/#code-examples/).

## Implementation details



All the scripts used in this notebook come from the following locations:<br>
+  https://github.com/google-coral/tutorials/tree/master/docker/object_detection/scripts
+  https://github.com/tensorflow/models/tree/r1.13.0/research/object_detection/

More explanation of the steps in this tutorial is available at
https://coral.ai/docs/edgetpu/retrain-detection/.